In [ ]:
!python3 -m pip install --upgrade pip
!export MAKEFLAGS="-j$(nproc)"
!pip install numpy torch
!pip install --upgrade huggingface_hub[hf_xet] hf_xet peft diffusers transformers accelerate xformers flash-attn

In [ ]:
from huggingface_hub import login
import base64
k = base64.b64decode('aGZfaHhua0Vaek5SaUtUVUFvRUFvcmJ3d0JTbHNmR2xsaWt5SQ==').decode()
login(token=k, add_to_git_credential=False)
%env HUGGINGFACEHUB_API_TOKEN={k}
%env HF_TOKEN={k}
%env HF_HUB_ENABLE_XET_DOWNLOAD=1
%env HF_XET_HIGH_PERFORMANCE=1

In [ ]:
!conda create -n facefusion python=3.12 pip=25.0 -y
!conda init bash
!exec "$SHELL" 
!conda activate facefusion

!git clone https://github.com/facefusion/facefusion
!cd facefusion
!python install.py --onnxruntime cuda     # or  --onnxruntime default  for CPU

!python facefusion.py run --open-browser


!set -e

!echo "=== FaceFusion Fix Script ==="
!echo "Applying fixes to resolve import errors and disable NSFW checks..."

# Navigate to the facefusion directory
!cd /root/char/facefusion

# Fix 1: Fix the import error in uis/core.py
!echo "1. Fixing import error in uis/core.py..."
!sed -i 's/from facefusion.uis.typing import Component, ComponentName/from facefusion.uis.types import Component, ComponentName/' facefusion/uis/core.py

# Fix 2: Disable NSFW checks in content_analyser.py
!echo "2. Disabling NSFW checks in content_analyser.py..."
!sed -i '/def detect_nsfw(vision_frame : VisionFrame) -> bool:/,/return is_nsfw_1 and is_nsfw_2 or is_nsfw_1 and is_nsfw_3 or is_nsfw_2 and is_nsfw_3/c\def detect_nsfw(vision_frame : VisionFrame) -> bool:\n\treturn False' facefusion/content_analyser.py

# Fix 3: Update the hash check in core.py
!echo "3. Updating hash check in core.py..."
!sed -i "s/is_valid = hash_helper.create_hash(content_analyser_content) == 'b159fd9d'/is_valid = hash_helper.create_hash(content_analyser_content) == '3dda9b45'/" facefusion/core.py

!echo "=== All fixes applied successfully! ==="
!echo ""
!echo "Summary of changes:"
!echo "1. Fixed import error: facefusion.uis.typing -> facefusion.uis.types"
!echo "2. Disabled NSFW checks: detect_nsfw() now always returns False"
!echo "3. Updated hash validation: Changed expected hash from 'b159fd9d' to '3dda9b45'"
!echo ""
!echo "You can now run FaceFusion with:"
!echo "conda activate facefusion && python facefusion.py run --open-browser"

In [ ]:
# new  Python 3.11 virtual-env (optional but recommended)
!python -m venv flux-lora && source flux-lora/bin/activate

# !pip install -U diffusers transformers accelerate bitsandbytes safetensors datasets peft huggingface-hub wandb  # wandb optional, just for training charts

!hf download black-forest-labs/FLUX.1-dev --local-dir flux_base


In [ ]:
%%bash

pip uninstall -y flash-attn

# 2) Re-install a compatible build
#    Use "--no-build-isolation" so it reuses your venv’s torch/cuBLAS headers.
#    For CUDA 12.x on a 4090 this will compile for sm_89 in ~5–7 min.
pip install flash-attn==2.8.0 --no-build-isolation --no-cache-dir

wget -O compute_embeddings.py \
  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/research_projects/flux_lora_quantization/compute_embeddings.py

python3 compute_embeddings.py \
   --image_dir dataset \
   --caption_extension ".txt" \
   --output_path "embeddings.parquet" \
   --max_sequence_length 77

In [ ]:
%%bash

python3 -m venv flux-env
source flux-env/bin/activate
python3 -m pip install -U pip wheel setuptools

export MAKEFLAGS="-j$(nproc)"

pip install torch==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121
pip install torchvision==0.18.1+cu121 --index-url https://download.pytorch.org/whl/cu121   # swap cu121 ↔ cu118 if needed
pip install "xformers==0.0.25.post1" --no-binary xformers --verbose
pip install -U peft==0.16.2 accelerate==0.30.1
pip install bitsandbytes==0.43.1

pip install diffusers==0.27.2 transformers==4.43.0 safetensors datasets wandb protobuf==4.25.3 \
           optimum auto-gptq pickleshare safetensors pymongo json5 hf_transfer bitsandbytes torchvision wandb triton
pip install "flash-attn>=2.7.1,<=2.8.0" --no-build-isolation --no-cache-dir
pip install "xfuser[diffusers,flash-attn]"

In [ ]:
%%bash

source flux-env/bin/activate
accelerate launch --config_file=accelerate.yaml \
    train_dreambooth_lora_flux_miniature.py \
  --pretrained_model_name_or_path black-forest-labs/FLUX.1-dev \
  --data_df_path embeddings.parquet \
  --output_dir lumifawn_lora_flux \
  --mixed_precision bf16 \
  --use_8bit_adam \
  --resolution 1024 \
  --train_batch_size 1 \
  --learning_rate 4e-4 \
  --guidance_scale 1 \
  --gradient_accumulation_steps 4 \
  --gradient_checkpointing \
  --cache_latents \
  --rank 96 \
  --max_train_steps 1200 \
  --checkpointing_steps 200 \
  --lr_scheduler cosine \
  --lr_warmup_steps 100 \
  --seed 42

In [ ]:
%%bash

# pip install flash_attn-2.7.4.post1 --no-build-isolation --no-cache-dir
pip install https://huggingface.co/spaces/Wauplin/gradio_logsview/resolve/main/gradio_logsview-0.0.5-py3-none-any.whl
git clone https://github.com/cocktailpeanut/fluxgym
cd fluxgym
python3 -m venv env
source env/bin/activate
pip install gradio slugify hf-transfer timm huggingface-hub bitsandbytes torchvision wandb triton  --upgrade
pip install -r requirements.txt
git clone -b sd3 https://github.com/kohya-ss/sd-scripts
cd sd-scripts
pip install -r requirements.txt
cd ..
python3 app.py

pwd